In [327]:
### RUNS A DASHBOARD IN THE BROWSER, NEED TO CLINK ON THE BLUE HTTP LINK TO SEE IT

import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import functools
import itertools
import operator
import os
import os.path as op
import numpy as np
import pytrends
from pytrends.request import TrendReq as UTrendReq
from datetime import date
import datetime as d
from collections import Counter
import click
import pdb
import sys, traceback

In [328]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px


In [329]:
# Opening JSON file
f = open('../data/raw/CMS_2010_to_June_2022_ENGLISH.json')
  
# returns JSON object as a dictionary
data = json.load(f)

# convert to data frame
df = pd.DataFrame.from_dict(data)

df['keywordStringsCleanAfterFuzz'] = df['keywordStrings']


In [330]:
df_clean = df[['id', 'lastModifiedDate', 'keywordStringsCleanAfterFuzz']]
df_clean.index = df_clean.index.astype('int64')

In [364]:
import plotly.graph_objects as go

def get_interest_over_time_plot(google_df,dw_mentions,selected_keyword):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
    x=google_df.resample('1W').mean()[selected_keyword].index,
    y=google_df.resample('1W').mean()[selected_keyword],
    name="Google Search Weekly Relative Average"       # this sets its legend entry
    ))


    fig.add_trace(go.Scatter(
    x=google_df.resample('1W').mean()[selected_keyword].index,
    y=dw_mentions,
    name="Number of DW articles per week"
    ))

    fig.update_layout(
    title="Plot Title",
    xaxis_title="Date",
    yaxis_title="Y Axis Title",
    legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"))

    fig.show()
    return fig



In [366]:
def get_dw_timeseries_FT(df_clean, keyword, start = '2019-01-01', end='2019-06-01'):

    not_keyword_indices = [] #TODO: do this without a loop (create extra boolean column and assign True if keyword is there)
    for i, row in enumerate(df_clean['keywordStringsCleanAfterFuzz']):
        if keyword not in row:
            not_keyword_indices.append(i)

    df_clean = df_clean.drop(not_keyword_indices)
    df_clean['lastModifiedDate'] = pd.to_datetime(df_clean['lastModifiedDate'])
    df_clean.set_index('lastModifiedDate', inplace = True)
    df_clean = df_clean.loc[start:end]
    output = df_clean.resample('1W').count()['id']
    return output

In [369]:
sys.path.append('..')

from models.MA_dw_to_google_pipeline import get_interest_over_time, get_dw_timeseries

app = Dash()

keyword_dropdown = dcc.Dropdown(options=['Germany', 'Donald Trump'], value='Germany')

app.layout = html.Div(children=[
    html.H1(children='Google Search Interest'),
    keyword_dropdown,
    dcc.Graph(id='search-graph')
])
@app.callback(
    Output(component_id='search-graph', component_property='figure'),
    Input(component_id=keyword_dropdown, component_property='value')
)
def update_graph(selected_keyword):
    google_df = get_interest_over_time(selected_keyword, start_date = '2019-01-01', end_date='2019-06-01')
    dw_mentions = get_dw_timeseries_FT(df_clean, selected_keyword, start = '2019-01-01', end='2019-06-01')
    line_fig = get_interest_over_time_plot(google_df,dw_mentions,selected_keyword)
    return line_fig

# Run local server
app.run_server(debug=True, port=8049, use_reloader=False)

Dash is running on http://127.0.0.1:8049/

 * Serving Flask app '__main__'
 * Debug mode: on
Germany


/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connec

Germany


/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connec